In [0]:
import json
import pandas as pd
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet.plot import plot_plotly, plot_components_plotly, add_changepoints_to_plot
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [0]:
OSFcast=spark.read.parquet('/mnt/MLPipeline/FcastCategory/')



In [0]:
#Convert to Pandas
pdf=OSFcast.toPandas()

In [0]:
pdf.shape

Out[37]: (324, 3)

In [0]:
dfa=pdf.copy()

In [0]:
x=dfa.groupby(['Category']).size().reset_index(name='count')



In [0]:
#You cant forecast where a category has less than 2 rows so filter them out
y=x[x['count'] <2]

In [0]:
y

Out[41]:

,Category,count


In [0]:
fil_os = y.Category.unique()

In [0]:
fil_os

Out[43]: array([], dtype=object)

In [0]:
os=['A','B','C']



In [0]:

df2=dfa[~dfa['Category'].isin(fil_os)]
df2=df2[df2['Category'].isin(os)]

In [0]:
df2

Out[46]:

,ds,Category,y
0,2013-01-01,C,994
1,2013-01-01,B,1058
2,2013-01-01,A,2748
3,2013-02-01,C,1411
4,2013-02-01,B,1488
...,...,...,...
319,2020-03-01,C,366065
320,2020-03-01,B,440819
321,2020-03-01,A,669643
322,2020-04-01,C,366065


In [0]:
#Prophet Model training in loop. It also saves all forecast results

list_os =df2.Category.unique()

fcsta_all = pd.DataFrame()
plots_a={}
components_a={}
for os in list_os:
    os_dfa = df2.loc[df2['Category'] == os]
    ma = Prophet(changepoint_prior_scale=0.09)
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    for i, month in enumerate(months):
        os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
        ma.add_regressor(month)

    ma.fit(os_dfa )
    future_a= ma.make_future_dataframe(periods=365)
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    for i, month in enumerate(months):
        future_a[month] = (future_a['ds'].dt.month == i + 1).values.astype('float')

    forecast_a= ma.predict(future_a)
    forecast_a['Category'] = os
    fcsta_all = pd.concat([fcsta_all, forecast_a], ignore_index=True)
    plots_a[os]=plot_plotly(ma, forecast_a)
    components_a[os] = plot_components_plotly(ma,forecast_a)

<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 os_dfa[month] = (os_dfa['ds'].dt.month == i + 1).values.astype('float')
<command-3749816432428012>:13: SettingWithCopyWarning: 
A value is trying to be 

In [0]:
for key, figure in plots_a.items():
    print (key)

C
B
A
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

In [0]:
# plt.style.use('ggplot')
# pd.plotting.register_matplotlib_converters()
# import plotly.io as pio
# pio.templates.default = "p"

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

In [0]:
for (key, figure),(key1,figure1) in zip(plots_a.items(),components_a.items()):
  fig=plots_a[key]
  fig.update_layout(title=key,autosize=False,
  width=1700,
  height=600,template='seaborn')  
  fig.show()
  fig1=components_a[key1]
  fig1.update_layout(title=key,autosize=False,
  width=1700,
  height=800,template ='plotly_dark')
  fig1.show()

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

INFO:py4j.java_gateway:Received command c on object id p0

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

INFO:py4j.java_gateway:Received command c on object id p0

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

In [0]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

# import plotly.graph_objs as go
# plt.style.use('ggplot')
# pd.plotting.register_matplotlib_converters()
# import plotly.io as pio
# pio.templates.default = "plotly_dark"


INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

In [0]:
import plotly.graph_objs as go
OName= sorted(set(fcsta_all['Category'].unique()))

fig=go.Figure()

os_plot_names = []
buttons=[]
default_os='A'
for os in OName:
    
    dff= fcsta_all.loc[fcsta_all['Category']==os]
    dfy= df2.loc[df2['Category']==os] 
    
    fig.add_trace(go.Scatter(
    name = 'predicted trend',
    mode = 'lines',
    x = list(dff['ds']),
    y = list(dff['yhat']),
    marker=dict(
        color='red',
        line=dict(width=3))))
    
    fig.add_trace(go.Scatter(
    name = 'upper band',
    mode = 'lines',
    x = list(dff['ds']),
    y = list(dff['yhat_upper']),
    line= dict(color='#57b88f'),
    fill = 'tonexty'))
  
    fig.add_trace(go.Scatter(
    name= 'lower band',
    mode = 'lines',
    x = list(dff['ds']),
    y = list(dff['yhat_lower']),
    line= dict(color='#1705ff')))
    
    fig.add_trace(go.Scatter(
    name = 'Actual',
    mode = 'markers',
    x = list(dfy['ds']),
    y = list(dfy['y']),
    marker=dict(
      color='pink',
      line=dict(width=2))))
        
    os_plot_names.extend([os]*4)
for os in OName:   
    buttons.append(dict(method='update',label=os,
                        args = [{'visible': [os==r for r in os_plot_names]}]))
    

fig.update_layout(showlegend=True,template='plotly_dark', updatemenus=[{"buttons": buttons,"direction": "down","showactive": True,"active": OName.index(default_os), "x": 0.5, "y": 1.15}])
fig.show()


INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0